<span style="font-family:cursive;font-size:30px;color:#622569;">Introduction</span>
     
<span style="font-family:serif;font-size:15px;color:#034f84;">Company ABC has a few thousand orders via their website on a daily basis and they have to deliver them as
fast as they can. For delivering the goods ordered by the customers, ABC has tied up with multiple courier companies in India who charge them some amount per delivery.
</span>

<span style="font-family:serif;font-size:15px;color:#034f84;">

The charges are dependent upon two factors:
- Weight of the product
- Distance between the warehouse (pickup location) and customer’s delivery address (destination location)
</span>
<span style="font-family:serif;font-size:15px;color:#034f84;">
On average, the delivery charges are Rs. 100 per shipment. So if ABC ships 1,00,000 orders per month, they have to pay approximately Rs. 1 crore to the courier companies on a monthly
basis as charges.
</span>
<span style="font-family:serif;font-size:15px;color:#034f84;">
As the amount that ABC has to pay to the courier companies is very high, they want to verify if the
charges levied by their partners per Order are correct.
</span>

<span style="font-family:cursive;font-size:25px;color:#e06377;">Import Libraries</span>

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import pandas_datareader as DataReader
from datetime import date


In [2]:
!pip install openpyxl

<span style="font-family:cursive; font-size:25px; color:crimson;"> Data Description </span> 

For this challenge you are given the 5 excel files which includes data from Company ABC and Courier Company.The goal of the analysis is to to verify if the
charges levied by their partners per Order are correct. 


<span style="font-family:cursive;font-size:25px;color:#622569;">Load DataSet</span>


In [3]:
df_pincodez_abc=pd.read_excel('../input/b2b-ecommerce-courier-dataset/Company ABC - Pincode Zones.xlsx',engine='openpyxl')
df_pincodez_abc.info()
df_pincodez_abc.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Warehouse Pincode  124 non-null    int64 
 1   Customer Pincode   124 non-null    int64 
 2   Zone               124 non-null    object
dtypes: int64(2), object(1)
memory usage: 3.0+ KB


,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d


In [4]:
rename_zone_abc=df_pincodez_abc.rename(columns={"Zone":"Delivery Zone As Per ABC"},inplace=True)
df_pincodez_abc.sample()

,Warehouse Pincode,Customer Pincode,Delivery Zone As Per ABC
100,121003,335803,b


In [5]:
df_courier_invoice=pd.read_excel('../input/b2b-ecommerce-courier-dataset/Courier Company - Invoice.xlsx',engine='openpyxl')
df_courier_invoice.info()
df_courier_invoice.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AWB Code              124 non-null    int64  
 1   Order ID              124 non-null    int64  
 2   Charged Weight        124 non-null    float64
 3   Warehouse Pincode     124 non-null    int64  
 4   Customer Pincode      124 non-null    int64  
 5   Zone                  124 non-null    object 
 6   Type of Shipment      124 non-null    object 
 7   Billing Amount (Rs.)  124 non-null    float64
dtypes: float64(2), int64(4), object(2)
memory usage: 7.9+ KB


,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4


In [6]:
df_courier_invoice.rename(columns = {"Zone":"Delivery Zone Charged By Courier Company","Charged Weight":"Total Weight As Per Courier Company (KG)"},inplace=True)
df_courier_invoice.sample()

,AWB Code,Order ID,Total Weight As Per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone Charged By Courier Company,Type of Shipment,Billing Amount (Rs.)
56,1091117616121,2001809592,1.5,121003,244713,b,Forward charges,89.6


In [7]:
df_order_report_abc=pd.read_excel('../input/b2b-ecommerce-courier-dataset/Company ABC- Order Report.xlsx',engine='openpyxl') 
df_order_report_abc.info()
df_order_report_abc.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ExternOrderNo  400 non-null    int64  
 1   SKU            400 non-null    object 
 2   Order Qty      400 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 9.5+ KB


,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


In [8]:
rename_order_id=df_order_report_abc.rename(columns={"ExternOrderNo":"Order ID"},inplace=True)
df_order_report_abc.head()

,Order ID,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


In [9]:
df_sku_inorder_master_abc=pd.read_excel('../input/b2b-ecommerce-courier-dataset/Company ABC - SKU Master.xlsx',engine='openpyxl')
df_sku_inorder_master_abc.info()
df_sku_inorder_master_abc.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SKU         66 non-null     object
 1   Weight (g)  66 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ KB


,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [10]:
df_courier_company_rates=pd.read_excel('../input/b2b-ecommerce-courier-dataset/Courier Company - Rates.xlsx',engine='openpyxl')
df_courier_company_rates.info()
df_courier_company_rates.describe()
df_courier_company_rates.sample()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fwd_a_fixed       1 non-null      float64
 1   fwd_a_additional  1 non-null      float64
 2   fwd_b_fixed       1 non-null      int64  
 3   fwd_b_additional  1 non-null      float64
 4   fwd_c_fixed       1 non-null      float64
 5   fwd_c_additional  1 non-null      float64
 6   fwd_d_fixed       1 non-null      float64
 7   fwd_d_additional  1 non-null      float64
 8   fwd_e_fixed       1 non-null      float64
 9   fwd_e_additional  1 non-null      float64
 10  rto_a_fixed       1 non-null      float64
 11  rto_a_additional  1 non-null      float64
 12  rto_b_fixed       1 non-null      float64
 13  rto_b_additional  1 non-null      float64
 14  rto_c_fixed       1 non-null      float64
 15  rto_c_additional  1 non-null      float64
 16  rto_d_fixed       1 non-null      float64
 17  r

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


<span style="font-family:cursive;font-size:25px;color:#622569;">EDA</span>

In [11]:
df_mg_sku_order=pd.merge(df_order_report_abc,df_sku_inorder_master_abc,how="left",on="SKU")

In [12]:
df_mg_sku_order.info()
df_mg_sku_order.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 401 entries, 0 to 400
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Order ID    401 non-null    int64  
 1   SKU         401 non-null    object 
 2   Order Qty   401 non-null    float64
 3   Weight (g)  401 non-null    int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 15.7+ KB


,Order ID,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001827036,8904223819093,1.0,150
2,2001827036,8904223819109,1.0,100
3,2001827036,8904223818430,1.0,165
4,2001827036,8904223819277,1.0,350


In [13]:
df_mg_sku_order.dropna()

,Order ID,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001827036,8904223819093,1.0,150
2,2001827036,8904223819109,1.0,100
3,2001827036,8904223818430,1.0,165
4,2001827036,8904223819277,1.0,350
...,...,...,...,...
396,2001806229,8904223818942,1.0,133
397,2001806229,8904223818850,1.0,240
398,2001806226,8904223818850,2.0,240
399,2001806210,8904223816214,1.0,120


In [14]:
df_mg_abc_courier = df_pincodez_abc.drop_duplicates(subset=['Customer Pincode'])
df_mg_courier_abc=df_courier_invoice[['Order ID', 'Customer Pincode','Type of Shipment']]
df_mg_pincodez=df_mg_courier_abc.merge(df_mg_abc_courier,on='Customer Pincode')
df_mg_pincodez.sample()

,Order ID,Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per ABC
5,2001806686,326502,Forward charges,121003,d


In [15]:
df_mg_sku_inorder=df_mg_sku_order.merge(df_mg_pincodez,on='Order ID')
df_mg_sku_inorder.sample()


,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per ABC
192,2001808992,8904223818850,1.0,240,342014,Forward charges,121003,b


In [16]:
kg=1000

In [17]:
df_re_mg_sku_inorder=df_mg_sku_inorder.rename(columns={"Order Qty":"Order_Qty","Weight (g)":"Weight_g"})

In [18]:
convert_g_to_kg="""
Total_Weight_Per_Shipment=(Order_Qty*Weight_g)/@kg
 """
df_re_mg_sku_inorder.eval(convert_g_to_kg,inplace=True,engine='python')

In [19]:
df_re_mg_sku_inorder['Weight_g'].value_counts()

240    98
127    59
133    41
112    40
120    22
350    19
210    15
165    15
121    12
100    11
115     8
290     8
65      7
113     5
137     5
500     5
140     4
130     4
600     4
232     3
10      3
150     2
110     2
170     2
490     1
160     1
250     1
102     1
30      1
300     1
552     1
Name: Weight_g, dtype: int64

In [20]:
df_re_mg_sku_inorder['Type of Shipment'].value_counts()

Forward charges            351
Forward and RTO charges     50
Name: Type of Shipment, dtype: int64

In [21]:
df_re_mg_sku_inorder

,Order ID,SKU,Order_Qty,Weight_g,Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per ABC,Total_Weight_Per_Shipment
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350
...,...,...,...,...,...,...,...,...,...
396,2001806229,8904223818942,1.0,133,421204,Forward charges,121003,d,0.133
397,2001806229,8904223818850,1.0,240,421204,Forward charges,121003,d,0.240
398,2001806226,8904223818850,2.0,240,723146,Forward charges,121003,d,0.480
399,2001806210,8904223816214,1.0,120,140604,Forward charges,121003,b,0.120


In [22]:
#renaming columns
re_df_mg_sku_inorder=df_re_mg_sku_inorder.rename(columns={"Order_Qty":"Order Qty","Total_Weight_Per_Shipment":"Total Weight AS Per ABC (KG)","Weight_g":"Weight (g)"})
re_df_mg_sku_inorder.sample()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per ABC,Total Weight AS Per ABC (KG)
266,2001807852,8904223818850,1.0,240,311001,Forward charges,121003,b,0.24


<span style="font-family:cursive;font-size:25px;color:#622569;">Function For Assigning Weight Slab</span>

In [23]:

def weight_slab(weight):
    i=round(weight%1,1)
    if i==0.0:
        return weight
    elif i>0.5:
        return int(weight)+1.0
    else:
        return int(weight)+0.5
  

<span style="font-family:cursive;font-size:25px;color:#622569;">Generation Of Weight Slab Column</span>

In [24]:
#Generation of Weight Slab
re_df_mg_sku_inorder['Weight Slab As Per ABC (KG)']=re_df_mg_sku_inorder['Total Weight AS Per ABC (KG)'].apply(weight_slab)
df_courier_invoice['Weight Slab Charged By Courier Company (KG)']=(df_courier_invoice['Total Weight As Per Courier Company (KG)']).apply(weight_slab)

In [25]:
re_df_mg_sku_inorder

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per ABC,Total Weight AS Per ABC (KG),Weight Slab As Per ABC (KG)
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5
...,...,...,...,...,...,...,...,...,...,...
396,2001806229,8904223818942,1.0,133,421204,Forward charges,121003,d,0.133,0.5
397,2001806229,8904223818850,1.0,240,421204,Forward charges,121003,d,0.240,0.5
398,2001806226,8904223818850,2.0,240,723146,Forward charges,121003,d,0.480,0.5
399,2001806210,8904223816214,1.0,120,140604,Forward charges,121003,b,0.120,0.5


In [26]:
df_courier_invoice

,AWB Code,Order ID,Total Weight As Per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone Charged By Courier Company,Type of Shipment,Billing Amount (Rs.),Weight Slab Charged By Courier Company (KG)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.5
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,1.0
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2.5
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,1.0
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.5
...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,1.0
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,0.5
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,0.5
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,0.5


<span style="font-family:cursive;font-size:25px;color:#622569;">Computation Of Expected Charge</span>

In [27]:
total_Expected_charge=[]
for i in range(len(re_df_mg_sku_inorder)):
    fwd_category='fwd_'+re_df_mg_sku_inorder['Delivery Zone As Per ABC'][i]
    fwd_fixed=df_courier_company_rates[fwd_category+'_fixed'][0]
    fwd_additional=df_courier_company_rates[fwd_category+'_additional'][0]
    rto_category='rto_'+re_df_mg_sku_inorder['Delivery Zone As Per ABC'][i]
    rto_fixed=df_courier_company_rates[rto_category+'_fixed'][0]
    rto_additional=df_courier_company_rates[rto_category+'_additional'][0]
    if re_df_mg_sku_inorder['Type of Shipment'][i]=='Forward charges':
        if re_df_mg_sku_inorder['Weight Slab As Per ABC (KG)'][i]<=0.5:
            total_Expected_charge.append(fwd_fixed)
        else:
            additional_weight=(re_df_mg_sku_inorder['Weight Slab As Per ABC (KG)'][i]-0.5)/0.5
            total_Expected_charge.append(fwd_fixed+additional_weight*fwd_additional)
    if re_df_mg_sku_inorder['Type of Shipment'][i]=='Forward and RTO charges':
        if re_df_mg_sku_inorder['Weight Slab As Per ABC (KG)'][i]<=0.5:
            total_Expected_charge.append(fwd_fixed+rto_fixed)
        else:
            additional_weight=(re_df_mg_sku_inorder['Weight Slab As Per ABC (KG)'][i]-0.5)/0.5
            total_Expected_charge.append(fwd_fixed+additional_weight*(fwd_additional+rto_additional))

re_df_mg_sku_inorder['Expected Charge As Per ABC (Rs.)']=total_Expected_charge


In [28]:
re_df_mg_sku_inorder.columns

Index(['Order ID', 'SKU', 'Order Qty', 'Weight (g)', 'Customer Pincode',
       'Type of Shipment', 'Warehouse Pincode', 'Delivery Zone As Per ABC',
       'Total Weight AS Per ABC (KG)', 'Weight Slab As Per ABC (KG)',
       'Expected Charge As Per ABC (Rs.)'],
      dtype='object')

In [29]:
Output2=re_df_mg_sku_inorder[['Order ID',
       'Type of Shipment', 'Delivery Zone As Per ABC',
       'Total Weight AS Per ABC (KG)', 'Weight Slab As Per ABC (KG)',
       'Expected Charge As Per ABC (Rs.)']]

In [30]:
df_courier_invoice.columns

Index(['AWB Code', 'Order ID', 'Total Weight As Per Courier Company (KG)',
       'Warehouse Pincode', 'Customer Pincode',
       'Delivery Zone Charged By Courier Company', 'Type of Shipment',
       'Billing Amount (Rs.)', 'Weight Slab Charged By Courier Company (KG)'],
      dtype='object')

In [31]:
Output1=df_courier_invoice[['AWB Code', 'Order ID','Total Weight As Per Courier Company (KG)',
        'Delivery Zone Charged By Courier Company','Billing Amount (Rs.)', 'Weight Slab Charged By Courier Company (KG)']]


In [32]:
df_merged_output=Output2.merge(Output1,on='Order ID',how='left')
df_merged_output.head(5)


,Order ID,Type of Shipment,Delivery Zone As Per ABC,Total Weight AS Per ABC (KG),Weight Slab As Per ABC (KG),Expected Charge As Per ABC (Rs.),AWB Code,Total Weight As Per Courier Company (KG),Delivery Zone Charged By Courier Company,Billing Amount (Rs.),Weight Slab Charged By Courier Company (KG)
0,2001827036,Forward charges,e,0.127,0.5,56.6,1091122418320,1.6,b,117.9,2.0
1,2001827036,Forward charges,e,0.150,0.5,56.6,1091122418320,1.6,b,117.9,2.0
2,2001827036,Forward charges,e,0.100,0.5,56.6,1091122418320,1.6,b,117.9,2.0
3,2001827036,Forward charges,e,0.165,0.5,56.6,1091122418320,1.6,b,117.9,2.0
4,2001827036,Forward charges,e,0.350,0.5,56.6,1091122418320,1.6,b,117.9,2.0


In [33]:
resultant_output1=df_merged_output.copy()

In [34]:
resultant_output1=resultant_output1.rename(columns={'AWB Code':'AWB Number','Billing Amount (Rs.)':'Charges_Billed_By_Courier_Company_Rs','Expected Charge As Per ABC (Rs.)':'Expected_Charge_As_Per_ABC_Rs'})
resultant_output1.head()


,Order ID,Type of Shipment,Delivery Zone As Per ABC,Total Weight AS Per ABC (KG),Weight Slab As Per ABC (KG),Expected_Charge_As_Per_ABC_Rs,AWB Number,Total Weight As Per Courier Company (KG),Delivery Zone Charged By Courier Company,Charges_Billed_By_Courier_Company_Rs,Weight Slab Charged By Courier Company (KG)
0,2001827036,Forward charges,e,0.127,0.5,56.6,1091122418320,1.6,b,117.9,2.0
1,2001827036,Forward charges,e,0.150,0.5,56.6,1091122418320,1.6,b,117.9,2.0
2,2001827036,Forward charges,e,0.100,0.5,56.6,1091122418320,1.6,b,117.9,2.0
3,2001827036,Forward charges,e,0.165,0.5,56.6,1091122418320,1.6,b,117.9,2.0
4,2001827036,Forward charges,e,0.350,0.5,56.6,1091122418320,1.6,b,117.9,2.0


<span style="font-family:cursive;font-size:25px;color:#622569;">Computation Of Difference Between Expected Charges And Billed Charges</span>

In [35]:
diff_bw_ec_bc="""
Difference_Between_Expected_Charges_and_Billed_Charges_Rs =(Expected_Charge_As_Per_ABC_Rs - Charges_Billed_By_Courier_Company_Rs)
"""
resultant_output1.eval(diff_bw_ec_bc,inplace=True,engine='python')

In [36]:
resultant_output1=resultant_output1.rename(columns={'Charges_Billed_By_Courier_Company_Rs':'Charges Billed By Courier Company (Rs.)','Expected_Charge_As_Per_ABC_Rs':'Expected Charge As Per ABC (Rs.)','Difference_Between_Expected_Charges_and_Billed_Charges_Rs':'Difference Between Expected Charges And Billed Charges (Rs.)'})


In [37]:
writer = pd.ExcelWriter('Expected_Result.xlsx',if_sheet_exists="replace",engine='openpyxl',mode="a")

In [38]:

resultant_output1.to_excel(writer,sheet_name='Calculations')
calculations=writer.save()

<span style="font-family:cursive;font-size:25px;color:#622569;">Computation Of Total Orders</span>

In [39]:
correctly_charged=resultant_output1[resultant_output1['Difference Between Expected Charges And Billed Charges (Rs.)']==0.0]
correctly=["Total Orders where ABC has been correctly charged",len(correctly_charged),sum(correctly_charged['Charges Billed By Courier Company (Rs.)'])]

over_charged=resultant_output1[resultant_output1['Difference Between Expected Charges And Billed Charges (Rs.)']<0.0]
over=["Total Orders where ABC has been over charged",len(over_charged),np.absolute(sum(over_charged['Difference Between Expected Charges And Billed Charges (Rs.)']))]

under_charged=resultant_output1[resultant_output1['Difference Between Expected Charges And Billed Charges (Rs.)']>0.0]
under=["Total Orders where ABC has been under charged",len(under_charged),np.absolute(sum(under_charged['Difference Between Expected Charges And Billed Charges (Rs.)']))]


In [40]:
resultant_output2=pd.DataFrame([correctly,over,under],columns=['Description','Count','Amount (Rs.)'])

In [41]:
resultant_output2

,Description,Count,Amount (Rs.)
0,Total Orders where ABC has been correctly charged,12,507.6
1,Total Orders where ABC has been over charged,380,31737.5
2,Total Orders where ABC has been under charged,9,165.2


<span style="font-family:cursive;font-size:25px;color:#622569;">Generation Of Resultant Summary Output File </span>

In [42]:
resultant_output2.to_excel(writer,sheet_name='Summary')
writer.save()

In [43]:
df_sheet_multi = pd.read_excel('Expected_Result.xlsx', sheet_name=['Calculations', 'Summary'])
df_sheet_multi['Calculations']


,Unnamed: 0,Order ID,Type of Shipment,Delivery Zone As Per ABC,Total Weight AS Per ABC (KG),Weight Slab As Per ABC (KG),Expected Charge As Per ABC (Rs.),AWB Number,Total Weight As Per Courier Company (KG),Delivery Zone Charged By Courier Company,Charges Billed By Courier Company (Rs.),Weight Slab Charged By Courier Company (KG),Difference Between Expected Charges And Billed Charges (Rs.)
0,0,2001827036,Forward charges,e,0.127,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
1,1,2001827036,Forward charges,e,0.150,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
2,2,2001827036,Forward charges,e,0.100,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
3,3,2001827036,Forward charges,e,0.165,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
4,4,2001827036,Forward charges,e,0.350,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,396,2001806229,Forward charges,d,0.133,0.5,45.4,1091117222080,0.71,d,90.2,1.0,-44.8
397,397,2001806229,Forward charges,d,0.240,0.5,45.4,1091117222080,0.71,d,90.2,1.0,-44.8
398,398,2001806226,Forward charges,d,0.480,0.5,45.4,1091117222065,0.68,d,90.2,1.0,-44.8
399,399,2001806210,Forward charges,b,0.120,0.5,33.0,1091117221940,2.92,b,174.5,3.0,-141.5


In [44]:
df_sheet_multi['Summary']

,Unnamed: 0,Description,Count,Amount (Rs.)
0,0,Total Orders where ABC has been correctly charged,12,507.6
1,1,Total Orders where ABC has been over charged,380,31737.5
2,2,Total Orders where ABC has been under charged,9,165.2
